In [8]:
import pandas as pd
import numpy as np
import psycopg2
from datetime import datetime

### DB connection

In [9]:
conn = psycopg2.connect(
    host="ec2-63-35-201-247.eu-west-1.compute.amazonaws.com",
    port="5432",
    user="postgres",
    password="Be6DkBT9FfY7CQ2vtj3Lpr6Wzqxax9",
    database="openremote"
)

def map_entity_id(entity_id):
    return idMap.get(entity_id, "no entity id found")


cur = conn.cursor()
idMap = {
    "4gBLwNljLBlc0oYpBdpGkD" : "ht_wp_1_id",
    "475wGcmfSE0L1wguboGfpW": "ht_wp_2_id",
    "43wZIInlpD6YFsEc8TkwlN": "weather",
    "30cjCSTbgsokSNalNoBbJh": "energie_prijs"
}

attribute_names = [
    'energyImportTotal',
    'temperature',
    'energy',
    'currentPrice'
]
batch_size = 500000
# Create a cursor object
cur = conn.cursor()


In [10]:
def fetch_batch(offset):
    query = f"""
    SELECT * FROM asset_datapoint 
    WHERE entity_id IN ({', '.join("'" + id + "'" for id in idMap.keys())})
    AND attribute_name IN ({', '.join("'" + name + "'" for name in attribute_names)})
    LIMIT {batch_size} OFFSET {offset};
    """
    cur.execute(query)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    return pd.DataFrame(rows, columns=columns)

# Function to get the total number of rows
def get_total_rows():
    query = f"""
    SELECT COUNT(*) FROM asset_datapoint 
     WHERE entity_id IN ({', '.join("'" + id + "'" for id in idMap.keys())})
    AND attribute_name IN ({', '.join("'" + id + "'" for id in attribute_names)})
    """
    cur.execute(query)
    total_rows = cur.fetchone()[0]
    return total_rows



In [11]:
# Get the total number of rows
total_rows = get_total_rows()
print(f"Total rows to process: {total_rows}")

# Calculate the number of batches
num_batches = (total_rows + batch_size - 1) // batch_size

Total rows to process: 559254


### Batch processing

In [12]:
processed_batches = []


# Process each batch
for i in range(num_batches):
    offset = i * batch_size
    print(f"Processing batch {i + 1}: OFFSET {offset}")

    # Fetch the batch
    batch_df = fetch_batch(offset)
    
    # Add new column
    batch_df['attributename_id'] = batch_df['entity_id'].apply(map_entity_id) + '_' +  batch_df['attribute_name']  
#     print(batch_df.columns)
    
    # Convert timestamp to datetime
#     batch_df['timestamp'] = pd.to_datetime(batch_df['timestamp'], format='mixed')
    batch_df['timestamp'] = pd.to_datetime(batch_df['timestamp'], errors='coerce')
    
    if batch_df['timestamp'].isnull().all():
        print(f"All timestamps in batch {i + 1} are NaT")
        continue
    
    # Pivot the DataFrame
    batch_pivoted_df = batch_df.pivot(index='timestamp', columns='attributename_id', values='value')
#     print(batch_pivoted_df.head())
    
    
    if batch_pivoted_df is None or batch_pivoted_df.empty:
        print(f"Pivoted DataFrame for batch {i + 1} is empty or None")
        continue
    # Reset index
#     batch_pivoted_df2 = batch_pivoted_df.reset_index('timestamp')

#     print(batch_pivoted_df.head())
    
    # Convert specified columns to numeric
#     for kolom in numeric_data:
#         if kolom in batch_pivoted_df.columns:
#             batch_pivoted_df[kolom] = pd.to_numeric(batch_pivoted_df[kolom], errors='coerce')
            
    for kolom in batch_pivoted_df.columns:
         batch_pivoted_df[kolom] = pd.to_numeric(batch_pivoted_df[kolom], errors='coerce')
        
    # Reset index to include 'timestamp' in the columns
#     batch_pivoted_df2 = batch_pivoted_df.reset_index('timestamp', inplace=True)
    batch_pivoted_df2 = batch_pivoted_df.reset_index('timestamp')
    
#     print(batch_pivoted_df2.head())
    
    
    # Add the processed batch to the list
    processed_batches.append(batch_pivoted_df2)

# Concatenate all processed batches into one DataFrame
processed_data = pd.concat(processed_batches)

Processing batch 1: OFFSET 0
Processing batch 2: OFFSET 500000


In [13]:
processed_data.head()

attributename_id,timestamp,energie_prijs_currentPrice,ht_wp_1_id_energyImportTotal,ht_wp_2_id_energyImportTotal,weather_temperature
0,2023-12-16 00:00:01.870,67.199997,NaN,NaN,NaN
1,2023-12-16 00:00:01.877,75.160000,NaN,NaN,NaN
2,2023-12-16 01:00:00.235,55.640000,NaN,NaN,NaN
3,2023-12-16 02:00:01.620,56.000000,NaN,NaN,NaN
4,2023-12-16 03:00:01.894,58.000000,NaN,NaN,NaN


In [14]:
processed_data.describe()

attributename_id,timestamp,energie_prijs_currentPrice,ht_wp_1_id_energyImportTotal,ht_wp_2_id_energyImportTotal,weather_temperature
count,510201,31008.000000,246447.000000,254466.000000,27333.000000
mean,2024-07-27 21:49:30.174397440,101.603677,189304.509436,311537.402769,13.079508
min,2023-12-16 00:00:01.870000,-200.000000,0.000000,0.000000,-2.990000
25%,2024-05-08 08:36:26.671000064,79.762500,174148.000000,296980.000000,7.360000
50%,2024-07-06 12:31:58.508000,100.500000,184194.000000,310904.000000,13.750000
75%,2024-10-20 14:50:46.105999872,123.580000,201090.000000,325785.000000,18.250000
max,2025-02-03 03:59:35.011000,550.000000,232291.000000,354302.000000,32.290000
std,NaN,48.231783,18740.934358,21169.367754,6.950413


In [15]:
print(processed_data.dtypes)

attributename_id
timestamp                       datetime64[ns]
energie_prijs_currentPrice             float64
ht_wp_1_id_energyImportTotal           float64
ht_wp_2_id_energyImportTotal           float64
weather_temperature                    float64
dtype: object
